### Author(s): evankoh


### Description
This notebook demonstrates:
1. How to retrieve historical prices for a stock
2. Compute moving averages
3. Plot results with line charts

### Import all neccessary libraries/functions

In [ ]:
import os
os.chdir('../')
from stockscafe.StocksCafeApi import StocksCafeApi
import stockscafe.indicators.momentum.MovingAverage as MA
import pandas as pd
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

### Parameters (If unfamiliar with python, then only change the values in this cell)

In [ ]:
exchange = 'SGX'
symbol = 'Z74'
name = 'Singtel'
shortMADays = 20
longMADays = 100
priceLookbackDays = 500 # Max is 1000

### Declaration & Get Recent Prices

In [ ]:
init_notebook_mode(connected=True)
sc = StocksCafeApi()
df = sc.getPrices(exchange, symbol, priceLookbackDays)
df.tail()

### Compute Moving Averages

In [ ]:
df = MA.compute(df, shortMADays) # Compute and add to Data Frame
df = MA.compute(df, longMADays) # Compute and add to Data Frame
df.tail() # Check data - Try 'df.head()'

### Plot Graph

In [ ]:
# Good resource to learn about graph basics - https://www.kaggle.com/kanncaa1/plotly-tutorial-for-beginners

traceClose = go.Scatter(
                    x = df.date,
                    y = df.close,
                    mode = "lines",
                    name = "Close",
                    marker = dict(color = 'rgba(16, 112, 2, 0.8)'))

traceShortMA = go.Scatter(
                    x = df.date,
                    y = df[f'sma{shortMADays}'],
                    mode = "lines",
                    name = f'SMA-{shortMADays}',
                    marker = dict(size = 10, color = 'rgba(0, 0, 255, 0.3)'))

traceLongMA = go.Scatter(
                    x = df.date,
                    y = df[f'sma{longMADays}'],
                    mode = "lines",
                    name = f'SMA-{longMADays}',
                    marker = dict(color = 'rgba(255, 0, 0, 0.3)'))

data = [traceClose, traceShortMA, traceLongMA]

layout = dict(title = f'{name} Moving Average',
              xaxis = dict(title = '', ticklen = 0, zeroline = False, 
                           showticklabels = True, range = [0, len(df.index)], dtick = 100, tickwidth = 1),
              autosize = True, showlegend = False
             )

fig = dict(data = data, layout = layout)

iplot(fig)

### Generate and store graph in html

In [ ]:
plot(fig, filename = f'output/{exchange}:{symbol}.html')